In [1]:
import numpy as np
from utils.slam_helpers import transform_to_frame,transformed_params2depthplussilhouette,transformed_params2rendervar,transformed_GRNparams2rendervar,transformed_GRNparams2depthplussilhouette
from diff_gaussian_rasterization import GaussianRasterizer as Renderer
from scripts.main_SurgeSplat import deform_gaussians, setup_camera
import torch
import os
from PIL import Image

System Paths:
/media/thesis_ssd/code/EndoGSLAM/EndoGSLAM
/media/thesis_ssd/anaconda/envs/endogslam/lib/python310.zip
/media/thesis_ssd/anaconda/envs/endogslam/lib/python3.10
/media/thesis_ssd/anaconda/envs/endogslam/lib/python3.10/lib-dynload

/media/thesis_ssd/anaconda/envs/endogslam/lib/python3.10/site-packages


In [2]:
intrinsics = torch.tensor([[199.6883,   0.0000, 166.3290],
        [  0.0000, 249.4753, 170.4058],
        [  0.0000,   0.0000,   1.0000]], device='cuda:0')
w2c = torch.tensor([[ 1.0000e+00,  6.5711e-11,  2.3283e-10,  0.0000e+00],
        [-3.1832e-11,  1.0000e+00, -7.4115e-21,  0.0000e+00],
        [-9.2644e-22,  2.9104e-11,  1.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]], device='cuda:0')

cam = setup_camera(336,336, intrinsics.cpu().numpy(), w2c.detach().cpu().numpy(), use_simplification=True)




In [3]:
params_np = np.load('/media/thesis_ssd/code/EndoGSLAM/EndoGSLAM/experiments/EndoNerf cutting_deform_short_simple/cutting_deform_short_simple/params.npz')
params={}
for key in params_np.keys():
    params[key] = torch.tensor(params_np[key]).cuda()

In [4]:
params

{'means3D': tensor([[[-6.8816, -6.8710, -6.8697,  ..., -6.8816, -6.8816, -6.8816],
          [-5.6433, -5.6351, -5.6424,  ..., -5.6433, -5.6433, -5.6433],
          [ 8.2618,  8.2693,  8.2660,  ...,  8.2618,  8.2618,  8.2618]],
 
         [[-6.6292, -6.6508, -6.6297,  ..., -6.6292, -6.6292, -6.6292],
          [-5.4692, -5.4460, -5.4606,  ..., -5.4692, -5.4692, -5.4692],
          [ 8.0069,  8.0243,  8.0073,  ...,  8.0069,  8.0069,  8.0069]],
 
         [[-6.5513, -6.5763, -6.5499,  ..., -6.5513, -6.5513, -6.5513],
          [-5.4378, -5.4014, -5.4401,  ..., -5.4378, -5.4378, -5.4378],
          [ 7.9610,  7.9857,  7.9674,  ...,  7.9610,  7.9610,  7.9610]],
 
         ...,
 
         [[ 5.6419,  5.6207,  5.6606,  ...,  5.5916,  5.5731,  5.6126],
          [ 4.4597,  4.4964,  4.4416,  ...,  4.5211,  4.5070,  4.4856],
          [ 6.7595,  6.7317,  6.7639,  ...,  6.7482,  6.7841,  6.7803]],
 
         [[ 5.7406,  5.7549,  5.7575,  ...,  5.7499,  5.7406,  5.7406],
          [ 4.5106,  4.51

In [5]:

# def deform_gaussians(params,time,deform_grad):

#     if deform_grad:
#         weights = params['deform_weights']
#         stds = params['deform_stds']
#         biases = params['deform_biases']
#     else:
#         weights = params['deform_weights'].detach()
#         stds = params['deform_stds'].detach()
#         biases = params['deform_biases'].detach()

#     deform = torch.sum(weights*torch.exp(-1/(2*stds**2)*(time-biases)**2),1) # Nx10 gaussians deformations
#     deform_xyz = deform[:,:3]
#     deform_rots = deform[:,3:7]
#     deform_scales = deform[:,7:10]
#     # print(f'xyz: {torch.sum(deform_xyz)}')
#     # print(torch.sum(deform_rots).item())
#     # print(torch.sum(deform_scales).item())
#     xyz = params['means3D']+deform_xyz
#     rots = params['unnorm_rotations']+deform_rots
#     scales = params['log_scales']+deform_scales

#     return xyz,rots,scales


# def deform_gaussians(params, time, deform_grad, N=5):
#     """
#     Calculate deformations using the N closest basis functions based on |time - bias|.

#     Args:
#         params (dict): Dictionary containing deformation parameters.
#         time (torch.Tensor): Current time step.
#         deform_grad (bool): Whether to calculate gradients for deformations.
#         N (int): Number of closest basis functions to consider.

#     Returns:
#         xyz (torch.Tensor): Updated 3D positions.
#         rots (torch.Tensor): Updated rotations.
#         scales (torch.Tensor): Updated scales.
#     """
#     if deform_grad:
#         weights = params['deform_weights']
#         stds = params['deform_stds']
#         biases = params['deform_biases']
#     else:
#         weights = params['deform_weights'].detach()
#         stds = params['deform_stds'].detach()
#         biases = params['deform_biases'].detach()

#     # Calculate the absolute difference between time and biases
#     time_diff = torch.abs(time - biases)

#     # Get the indices of the N smallest time differences
#     _, top_indices = torch.topk(-time_diff, N, dim=1)  # Negative for smallest values

#     # Create a mask to select only the top N basis functions
#     mask = torch.zeros_like(time_diff, dtype=torch.float)
#     mask.scatter_(1, top_indices, 1.0)

#     # Apply the mask to weights and biases
#     masked_weights = weights * mask
#     masked_biases = biases * mask

#     # Calculate deformations
#     deform = torch.sum(
#         masked_weights * torch.exp(-1 / (2 * stds**2) * (time - masked_biases)**2), dim=1
#     )  # Nx10 gaussians deformations

#     deform_xyz = deform[:, :3]
#     deform_rots = deform[:, 3:7]
#     deform_scales = deform[:, 7:10]

#     xyz = params['means3D'] + deform_xyz
#     rots = params['unnorm_rotations'] + deform_rots
#     scales = params['log_scales'] + deform_scales

#     return xyz, rots, scales

def deform_gaussians(params, time, deform_grad, N=5,deformation_type = 'gaussian'):
    """
    Calculate deformations using the N closest basis functions based on |time - bias|.

    Args:
        params (dict): Dictionary containing deformation parameters.
        time (torch.Tensor): Current time step.
        deform_grad (bool): Whether to calculate gradients for deformations.
        N (int): Number of closest basis functions to consider.

    Returns:
        xyz (torch.Tensor): Updated 3D positions.
        rots (torch.Tensor): Updated rotations.
        scales (torch.Tensor): Updated scales.
    """
    if deformation_type =='gaussian':
        if True:
            if deform_grad:
                weights = params['deform_weights']
                stds = params['deform_stds']
                biases = params['deform_biases']
            else:
                weights = params['deform_weights'].detach()
                stds = params['deform_stds'].detach()
                biases = params['deform_biases'].detach()

            # Calculate the absolute difference between time and biases
            time_diff = torch.abs(time - biases)

            # Get the indices of the N smallest time differences
            _, top_indices = torch.topk(-time_diff, N, dim=1)  # Negative for smallest values

            # Create a mask to select only the top N basis functions
            mask = torch.zeros_like(time_diff, dtype=torch.float)
            mask.scatter_(1, top_indices, 1.0)

            # Apply the mask to weights and biases
            masked_weights = weights * mask
            masked_biases = biases * mask

            # Calculate deformations
            deform = torch.sum(
                masked_weights * torch.exp(-1 / (2 * stds**2) * (time - masked_biases)**2), dim=1
            )  # Nx10 gaussians deformations

            deform_xyz = deform[:, :3]
            deform_rots = deform[:, 3:7]
            deform_scales = deform[:, 7:10]
        else:
            
            if deform_grad:
                weights = params['deform_weights']
                stds = params['deform_stds']
                biases = params['deform_biases']
            else:
                weights = params['deform_weights'].detach()
                stds = params['deform_stds'].detach()
                biases = params['deform_biases'].detach()

            # Calculate the absolute difference between time and biases
            time_diff = torch.abs(time - biases)

            # Get the indices of the N smallest time differences
            _, top_indices = torch.topk(-time_diff, N, dim=1)  # Negative for smallest values

            # Create a mask to select only the top N basis functions
            mask = torch.zeros_like(time_diff, dtype=torch.float)
            mask.scatter_(1, top_indices, 1.0).detach()

            # Register a gradient hook to zero out gradients for irrelevant basis functions
            if deform_grad:
                def zero_out_irrelevant_gradients(grad):
                    return grad * mask

                weights.register_hook(zero_out_irrelevant_gradients)
                biases.register_hook(zero_out_irrelevant_gradients)
                stds.register_hook(zero_out_irrelevant_gradients)

            # Calculate deformations
            deform = torch.sum(
                weights * torch.exp(-1 / (2 * stds**2) * (time - biases)**2), dim=1
            )  # Nx10 gaussians deformations

            deform_xyz = deform[:, :3]
            deform_rots = deform[:, 3:7]
            deform_scales = deform[:, 7:10]

        xyz = params['means3D'] + deform_xyz
        rots = params['unnorm_rotations'] + deform_rots
        scales = params['log_scales'] + deform_scales


    elif deformation_type == 'simple':
        with torch.no_grad():
            xyz = params['means3D'][...,time]
            rots = params['unnorm_rotations'][...,time]
            scales = params['log_scales'][...,time]

    return xyz, rots, scales

for id in range(params['cam_unnorm_rots'].shape[-1]):
    local_means,local_rots,local_scales = deform_gaussians(params,id,deform_grad = True,deformation_type='simple')


    #  print(torch.sum(local_means-params['means3D']))

    transformed_pts = transform_to_frame(local_means,params,id,False,False)





    # Initialize Render Variables
    rendervar = transformed_GRNparams2rendervar(params, transformed_pts,local_rots,local_scales)
    print(local_scales.max())
    rv_store = {}
    for key in rendervar.keys():
        rv_store[key] = rendervar[key].cpu().detach()
        local_means_store = local_means.cpu()
        local_scales_store = local_rots.cpu()
        local_rots_store = local_rots.cpu()
        transformed_pts_store = transformed_pts.cpu()



    #  rendervar['means3D'].retain_grad()
    depth_sil_rendervar = transformed_GRNparams2depthplussilhouette(params, w2c,
                                            transformed_pts,local_rots,local_scales)


    #RGB Rendering

    rendervar['means2D'].retain_grad()
    im, radius, _ = Renderer(raster_settings=cam)(**rendervar)
    # variables['means2D'] = rendervar['means2D'] # Gradient only accum from colour render for densification
    img = Image.fromarray((im.permute(1,2,0).cpu().detach().numpy()*255).astype(np.uint8))
    os.makedirs(f'./eval_plots/plots_simple/',exist_ok=True)
    img.save(f'./eval_plots/plots_simple/{id}.png')

    print(local_means)


tensor(0.0018, device='cuda:0')
tensor([[-6.8816, -5.6433,  8.2618],
        [-6.6292, -5.4692,  8.0069],
        [-6.5513, -5.4378,  7.9610],
        ...,
        [ 5.6419,  4.4597,  6.7595],
        [ 5.7406,  4.5106,  6.8367],
        [ 6.1184,  4.7790,  7.2436]], device='cuda:0')
tensor(0.0018, device='cuda:0')
tensor([[-6.8710, -5.6351,  8.2693],
        [-6.6508, -5.4460,  8.0243],
        [-6.5763, -5.4014,  7.9857],
        ...,
        [ 5.6207,  4.4964,  6.7317],
        [ 5.7549,  4.5106,  6.8285],
        [ 6.1269,  4.7814,  7.2390]], device='cuda:0')
tensor(0.0018, device='cuda:0')
tensor([[-6.8697, -5.6424,  8.2660],
        [-6.6297, -5.4606,  8.0073],
        [-6.5499, -5.4401,  7.9674],
        ...,
        [ 5.6606,  4.4416,  6.7639],
        [ 5.7575,  4.4934,  6.8228],
        [ 6.1203,  4.7728,  7.2388]], device='cuda:0')
tensor(0.0018, device='cuda:0')
tensor([[-6.8771, -5.6386,  8.2607],
        [-6.6298, -5.4665,  8.0101],
        [-6.5456, -5.4353,  7.9614],
  

In [6]:
# import matplotlib.pyplot as plt
# weights =   params['deform_weights'].cpu().detach()
# biases =    params['deform_biases'].cpu().detach()
# stds =      params['deform_stds'].cpu().detach()

# deforms = []
# deformsx1 = []
# for time in range(100):
#     deform = torch.sum(weights*torch.exp(-1/(2*stds**2)*(time-biases)**2),1)
#     deforms.append(deform) # Nx10 gaussians deformations
#     deformsx1.append(deform[0,0])
# fig,ax = plt.subplots(10,10,figsize = (25,25),sharey=True)
# for i in range(10):
#     for ii in range(10):
#         ax[ii,i].plot([deforms[idx][i+91720//3,ii] for idx in range(len(deforms))])
#         ax[ii,i].axhline(y = 0.0,color = 'r', linestyle = '--')    

In [7]:
# from GRN.models.conv_unet import GaussianRegressionNetwork
# import torch

# state_dict = torch.load('logs/GRN_6/checkpoint.pth',weights_only=False)
# model_state_dict = state_dict['model']

# model = GaussianRegressionNetwork()
# # model.load_state_dict(model_state_dict)

# torch.save(model.state_dict(),'GRN/models/GRN_v1.pth')
